## The functionalities in this notebook is as follows:

1. Given a graph, calculate the kernel matrix - diffusion for example

2. Sample from a GP with this kernel matrix, for a given hyper parameters

3. fit another GP with the same kernel  (and unknown hyperparamters)

### Example: Sample data from a GP on graph

### Taylor Expansion of the Diffusion Kernel

In [1]:
import numpy as np

def get_laplacian_matrix(W):
    "To get the graph Laplacian from adj matrix, symmetric adjacency matrix is assumed."
    nb_vertices = len(W)
    L = 0.5 * np.eye(nb_vertices)
    degrees = np.sum(W,axis=1)
    for i in range(nb_vertices):
        for j in range(i):
            L[i,j] = - W[i,j]/np.sqrt(degrees[i] * degrees[j])
    L += L.T
    return L

In [2]:
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.linalg import expm
from ipywidgets import FloatSlider, IntSlider, interactive

def get_laplacian_matrix(W):
    "To get the graph Laplacian from adj matrix, symmetric adjacency matrix is assumed."
    nb_vertices = len(W)
    L = 0.5 * np.eye(nb_vertices)
    degrees = np.sum(W,axis=1)
    for i in range(nb_vertices):
        for j in range(i):
            L[i,j] = - W[i,j]/np.sqrt(degrees[i] * degrees[j])
    L += L.T
    return L

def diffusion_kernel(adj_matrix, beta):
    """Compute the diffusion kernel using matrix exponential."""
    laplacian = get_laplacian_matrix(adj_matrix)
    return expm(-beta * laplacian)

def taylor_expansion_diffusion_kernel(adj_matrix, beta, num_terms=11):
    """Compute the Taylor expansion of the diffusion kernel."""
    laplacian = get_laplacian_matrix(adj_matrix)
    K = np.eye(adj_matrix.shape[0])  # Initialize kernel matrix
    for i in range(1, num_terms):
        K += (-beta)**i * np.linalg.matrix_power(laplacian, i) / math.factorial(i)

    # Check for positive definiteness by ensuring diagonal dominance
    min_diagonal = np.min(np.diag(K))
    if min_diagonal <= 0:
        adjustment = -min_diagonal + 1e-6  # Adding a small epsilon for strict positivity
        K += adjustment * np.eye(adj_matrix.shape[0])

    return K


def sample_from_gp(kernel):
    """Sample from a Gaussian process given a kernel matrix."""
    np.random.seed(42)
    L = np.linalg.cholesky(kernel + 1e-6 * np.eye(kernel.shape[0]))
    rv_matrix = np.random.normal(size=(kernel.shape[0], 1))
    return L @ rv_matrix

def plot_samples_and_kernels(beta, num_terms, num_nodes=20):
    """Plot sampled values and kernel matrices."""
    adjacency_matrix = np.eye(num_nodes, k=1) + np.eye(num_nodes, k=-1)  # Circular adjacency matrix
    K = diffusion_kernel(adjacency_matrix, beta)
    K_taylor = taylor_expansion_diffusion_kernel(adjacency_matrix, beta, num_terms)
    
    samples = sample_from_gp(K)
    samples_taylor = sample_from_gp(K_taylor)
    
    # Calculate the Frobenius norm of the difference
    frobenius_norm = np.linalg.norm(K - K_taylor, 'fro')

    # Create a figure with 2x2 subplots
    fig, axs = plt.subplots(2, 2, figsize=(12, 12))

    def plot_sampled_values(ax, samples, title, color):
        ax.plot(range(num_nodes), samples.flatten(), marker='o', linestyle='-', color=color)
        ax.set_title(title)
        ax.set_xlabel('Node Number')
        ax.set_ylabel('Sampled Value')
        ax.set_xticks(range(num_nodes))
        ax.grid()

    def plot_heatmap(ax, matrix, title):
        cax = ax.matshow(matrix, cmap='viridis')
        ax.set_title(title)
        plt.colorbar(cax, ax=ax)
        ax.set_xlabel('Node Index')
        ax.set_ylabel('Node Index')

    # Plot sampled values and heatmaps
    plot_sampled_values(axs[0, 0], samples, f'Sampled Values (beta={beta:.2f})', 'b')
    plot_heatmap(axs[0, 1], K, f'Kernel Matrix Heatmap (beta={beta:.2f})')
    plot_sampled_values(axs[1, 0], samples_taylor, f'Sampled Values for Taylor Expansion (beta={beta:.2f})', 'r')
    plot_heatmap(axs[1, 1], K_taylor, f'Taylor Expansion Heatmap (beta={beta:.2f})\nFrobenius Norm: {frobenius_norm:.4f}')

    plt.tight_layout()
    plt.show()

# Interactive sliders for beta and num_terms
beta_slider = FloatSlider(value=2.0, min=0.1, max=10.0, step=0.1, description='Beta (σ):')
num_terms_slider = IntSlider(value=11, min=1, max=20, step=1, description='Num Terms:')
interactive_plot = interactive(plot_samples_and_kernels, beta=beta_slider, num_terms=num_terms_slider)
interactive_plot


interactive(children=(FloatSlider(value=2.0, description='Beta (σ):', max=10.0, min=0.1), IntSlider(value=11, …

### Visualizing the (linear / random) graph we fit and the fitted GP

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm
import gpflow
import tensorflow as tf
import ipywidgets as widgets
from IPython.display import display, clear_output
import networkx as nx

def get_laplacian_matrix(adj_matrix):
    """Compute the Laplacian matrix for the given adjacency matrix."""
    D = np.diag(np.sum(adj_matrix, axis=1))  # Degree matrix
    return D - adj_matrix  # Laplacian matrix

def diffusion_kernel(adj_matrix, beta):
    """Compute the diffusion kernel using matrix exponential."""
    laplacian = get_laplacian_matrix(adj_matrix)
    return expm(-beta * laplacian)

# Parameters
num_nodes = 100
# Generate an undirected random graph
probability = 0.03  # Probability of edge creation
G = nx.erdos_renyi_graph(num_nodes, probability, directed=False)  # Ensure the graph is undirected
adjacency_matrix = nx.to_numpy_array(G)  # Convert to adjacency matrix

# Generate noisy samples function
def generate_noisy_samples(beta_sample):
    K = diffusion_kernel(adjacency_matrix, beta_sample)
    L = np.linalg.cholesky(K + 1e-6 * np.eye(num_nodes))  # Cholesky decomposition
    true_samples = L @ np.random.normal(size=(num_nodes, 1))  # Sample from Gaussian process
    noise = 0.1 * np.random.randn(num_nodes, 1)  # Additive noise
    Y_noisy = true_samples + noise  # Noisy observations
    return Y_noisy

# Modified `K` function in GraphDiffusionKernel class
class GraphDiffusionKernel(gpflow.kernels.Kernel):
    def __init__(self, adjacency_matrix, **kwargs):
        super().__init__(**kwargs)
        self.adjacency_matrix = tf.convert_to_tensor(adjacency_matrix, dtype=tf.float64)
        self.beta = gpflow.Parameter(2.0, transform=gpflow.utilities.positive())  # Learnable hyperparameter

    def K(self, X1, X2=None):
        if X2 is None:
            X2 = X1
        # Compute the full diffusion kernel
        kernel_matrix = self.diffusion_kernel(self.adjacency_matrix, self.beta)
        indices_X1 = tf.cast(tf.reshape(X1, [-1]), dtype=tf.int32)
        indices_X2 = tf.cast(tf.reshape(X2, [-1]), dtype=tf.int32)
        return tf.gather(tf.gather(kernel_matrix, indices_X1, axis=0), indices_X2, axis=1)

    def K_diag(self, X):
        kernel_matrix = self.diffusion_kernel(self.adjacency_matrix, self.beta)
        indices_X = tf.cast(tf.reshape(X, [-1]), dtype=tf.int32)
        return tf.gather(tf.linalg.diag_part(kernel_matrix), indices_X)
    
    def get_laplacian_matrix(self, W):
        """Get the graph Laplacian from the adjacency matrix."""
        nb_vertices = len(W)
        L = 0.5 * np.eye(nb_vertices)
        degrees = np.sum(W, axis=1)
        for i in range(nb_vertices):
            for j in range(i):
                L[i, j] = -W[i, j] / np.sqrt(degrees[i] * degrees[j])
        L += L.T
        return L

    def diffusion_kernel(self, adj_matrix, beta):
        laplacian = get_laplacian_matrix(adj_matrix)  # Graph Laplacian
        return tf.linalg.expm(-beta * laplacian)

# Function to plot the results
def plot_results(beta_sample):
    clear_output(wait=True)  # Clear previous output
    # Generate noisy samples
    Y_noisy = generate_noisy_samples(beta_sample)

    # Convert noisy data to TensorFlow tensor
    X = tf.convert_to_tensor(np.arange(num_nodes, dtype=np.float64).reshape(-1, 1))  # Input features (nodes)
    Y = tf.convert_to_tensor(Y_noisy, dtype=tf.float64)  # Noisy sampled data

    # Create an instance of the kernel
    graph_kernel = GraphDiffusionKernel(adjacency_matrix)

    # GPflow model
    model = gpflow.models.GPR(data=(X, Y), kernel=graph_kernel, mean_function=None)

    # Optimize the model
    gpflow.optimizers.Scipy().minimize(model.training_loss, model.trainable_variables)

    # Prediction for visualization
    X_new = tf.convert_to_tensor(np.arange(num_nodes).reshape(-1, 1), dtype=tf.float64)  # New input features for prediction
    mean, variance = model.predict_f(X_new)  # Predict mean and variance
    stddev = tf.sqrt(variance)  # Standard deviation

    # Create side-by-side plots for GP and network graph
    fig, ax = plt.subplots(1, 2, figsize=(16, 6))

    # Gaussian Process plot on the left
    ax[0].plot(X.numpy(), Y.numpy(), 'ro', label='Noisy Samples')
    ax[0].plot(X_new.numpy(), mean.numpy(), 'b-', label='Fitted Mean')
    ax[0].fill_between(X_new.numpy().flatten(),
                       (mean - 1.96 * stddev).numpy().flatten(),
                       (mean + 1.96 * stddev).numpy().flatten(),
                       color='lightblue', alpha=0.5, label='95% Confidence Interval')
    ax[0].set_title(f'Gaussian Process Fit with Graph Diffusion Kernel (beta={beta_sample:.2f})')
    ax[0].set_xlabel('Node Number')
    ax[0].set_ylabel('Sampled Value')
    ax[0].grid()
    ax[0].legend()
    
    # Network graph plot on the right
    plot_network_graph(adjacency_matrix, ax[1])

    plt.show()
    print("Learned beta:", model.kernel.beta.numpy())

# Function to plot the network graph in Matplotlib
def plot_network_graph(adjacency_matrix, ax):
    # Create a NetworkX graph from the adjacency matrix
    G = nx.from_numpy_array(adjacency_matrix)
    pos = nx.spring_layout(G)
    
    # Draw nodes
    nx.draw_networkx_nodes(G, pos, ax=ax, node_size=300, node_color='red')
    
    # Draw edges
    nx.draw_networkx_edges(G, pos, ax=ax, edge_color='blue')
    
    ax.set_title("Network Graph Representation of Adjacency Matrix")
    ax.axis("off")

# Create a slider for beta
beta_slider = widgets.FloatSlider(value=1.0, min=0.05, max=2.0, step=0.05, description='Beta:')
ui = widgets.VBox([beta_slider])

# Link the slider to the plotting function
out = widgets.interactive_output(plot_results, {'beta_sample': beta_slider})
display(ui, out)


Output()

In [22]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm
import gpflow
import tensorflow as tf
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.graph_objs as go
import networkx as nx

def get_laplacian_matrix(W):
    "To get the graph Laplacian from adj matrix, symmetric adjacency matrix is assumed."
    nb_vertices = len(W)
    L = 0.5 * np.eye(nb_vertices)
    degrees = np.sum(W,axis=1)
    for i in range(nb_vertices):
        for j in range(i):
            L[i,j] = - W[i,j]/np.sqrt(degrees[i] * degrees[j])
    L += L.T
    return L

# Function to compute the diffusion kernel
def diffusion_kernel(adj_matrix, beta):
    laplacian = get_laplacian_matrix(adj_matrix)  # Graph Laplacian
    return expm(-beta * laplacian)  # Matrix exponential

# Parameters
num_nodes = 100
adjacency_matrix = np.eye(num_nodes, k=1) + np.eye(num_nodes, k=-1)  # Circular adjacency matrix

# Generate noisy samples function
def generate_noisy_samples(beta_sample):
    K = diffusion_kernel(adjacency_matrix, beta_sample)
    L = np.linalg.cholesky(K + 1e-6 * np.eye(num_nodes))  # Cholesky decomposition
    true_samples = L @ np.random.normal(size=(num_nodes, 1))  # Sample from Gaussian process
    noise = 0.05 * np.random.randn(num_nodes, 1)  # Additive noise
    Y_noisy = true_samples + noise  # Noisy observations
    return Y_noisy

class TaylorExpansionDiffusionKernel(gpflow.kernels.Kernel):
    def __init__(self, adjacency_matrix, num_terms=15, **kwargs):
        super().__init__(**kwargs)
        self.adjacency_matrix = tf.convert_to_tensor(adjacency_matrix, dtype=tf.float64)
        self.beta = gpflow.Parameter(2.0, transform=gpflow.utilities.positive())  # Learnable hyperparameter
        self.num_terms = num_terms  # Number of terms for Taylor expansion

    def K(self, X1, X2=None):
        if X2 is None:
            X2 = X1
        # Compute the Taylor expansion kernel matrix
        kernel_matrix = self.taylor_expansion_diffusion_kernel(self.adjacency_matrix, self.beta)
        indices_X1 = tf.cast(tf.reshape(X1, [-1]), dtype=tf.int32)
        indices_X2 = tf.cast(tf.reshape(X2, [-1]), dtype=tf.int32)
        return tf.gather(tf.gather(kernel_matrix, indices_X1, axis=0), indices_X2, axis=1)

    def K_diag(self, X):
        kernel_matrix = self.taylor_expansion_diffusion_kernel(self.adjacency_matrix, self.beta)
        indices_X = tf.cast(tf.reshape(X, [-1]), dtype=tf.int32)
        return tf.gather(tf.linalg.diag_part(kernel_matrix), indices_X)

    def get_laplacian_matrix(self, W):
        """Get the graph Laplacian from the adjacency matrix."""
        W = np.array(W)  # Ensure W is a numpy array
        nb_vertices = len(W)
        L = np.eye(nb_vertices) * np.sum(W, axis=1)
        for i in range(nb_vertices):
            for j in range(i):
                if W[i, j] > 0:
                    L[i, j] = L[j, i] = -W[i, j] / np.sqrt(np.sum(W[i]) * np.sum(W[j]))
        return L

    def taylor_expansion_diffusion_kernel(self, adj_matrix, beta):
        """Compute the Taylor expansion of the diffusion kernel."""
        laplacian = self.get_laplacian_matrix(adj_matrix)
        K = np.eye(adj_matrix.shape[0])  # Initialize kernel matrix
        for i in range(1, self.num_terms):
            K += (-beta)**i * np.linalg.matrix_power(laplacian, i) / math.factorial(i)
        
        # K += 1e-2 * np.eye(adj_matrix.shape[0]) # Adding a small epsilon for numerical stability

        return tf.convert_to_tensor(K, dtype=tf.float64)  # Convert to TensorFlow tensor

# Function to plot the results
def plot_results(beta_sample):
    clear_output(wait=True)  # Clear previous output
    # Generate noisy samples
    Y_noisy = generate_noisy_samples(beta_sample)

    # Convert noisy data to TensorFlow tensor
    X = tf.convert_to_tensor(np.arange(num_nodes, dtype=np.float64).reshape(-1, 1))  # Input features (nodes)
    Y = tf.convert_to_tensor(Y_noisy, dtype=tf.float64)  # Noisy sampled data

    # Create an instance of the kernel
    graph_kernel = TaylorExpansionDiffusionKernel(adjacency_matrix)

    # GPflow model
    model = gpflow.models.GPR(data=(X, Y), kernel=graph_kernel, mean_function=None)

    # Optimize the model
    gpflow.optimizers.Scipy().minimize(model.training_loss, model.trainable_variables)

    # Prediction for visualization
    X_new = tf.convert_to_tensor(np.arange(num_nodes).reshape(-1, 1), dtype=tf.float64)  # New input features for prediction
    mean, variance = model.predict_f(X_new)  # Predict mean and variance
    print('Average Variance: ',variance.numpy().mean())
    stddev = tf.sqrt(variance)  # Standard deviation

    # Plotting the Gaussian Process results
    fig, ax = plt.subplots(1, 2, figsize=(16, 6))

    # GP plot on the left
    ax[0].plot(X.numpy(), Y.numpy(), 'ro', label='Noisy Samples')
    ax[0].plot(X_new.numpy(), mean.numpy(), 'b-', label='Fitted Mean')
    ax[0].fill_between(X_new.numpy().flatten(),
                       (mean - 1.96 * stddev).numpy().flatten(),
                       (mean + 1.96 * stddev).numpy().flatten(),
                       color='lightblue', alpha=0.5, label='95% Confidence Interval')
    ax[0].set_title(f'Gaussian Process Fit with Graph Diffusion Kernel (beta={beta_sample})')
    ax[0].set_xlabel('Node Number')
    ax[0].set_ylabel('Sampled Value')
    ax[0].grid()
    ax[0].legend()
    
    # Plot the network graph using NetworkX and Plotly
    plot_network_graph(adjacency_matrix, ax[1])

    plt.show()
    print("Learned beta:", model.kernel.beta.numpy())

# Function to plot the network graph in Matplotlib
def plot_network_graph(adjacency_matrix, ax):
    # Create a NetworkX graph from the adjacency matrix
    G = nx.from_numpy_array(adjacency_matrix)
    pos = nx.spring_layout(G)
    
    # Draw nodes
    nx.draw_networkx_nodes(G, pos, ax=ax, node_size=300, node_color='red')
    
    # Draw edges
    nx.draw_networkx_edges(G, pos, ax=ax, edge_color='blue')
    
    ax.set_title("Network Graph Representation of Adjacency Matrix")
    ax.axis("off")

# Create a slider for beta
beta_slider = widgets.FloatSlider(value=0.3, min=0.05, max=2.0, step=0.05, description='Beta:')
ui = widgets.VBox([beta_slider])

# Link the slider to the plotting function
out = widgets.interactive_output(plot_results, {'beta_sample': beta_slider})
display(ui, out)


Output()

In [18]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm
import gpflow
import tensorflow as tf
import ipywidgets as widgets
from IPython.display import display, clear_output
import networkx as nx

def get_laplacian_matrix(W):
    "To get the graph Laplacian from adj matrix, symmetric adjacency matrix is assumed."
    nb_vertices = len(W)
    L = 0.5 * np.eye(nb_vertices)
    degrees = np.sum(W,axis=1)
    for i in range(nb_vertices):
        for j in range(i):
            L[i,j] = - W[i,j]/np.sqrt(degrees[i] * degrees[j])
    L += L.T
    return L

def diffusion_kernel(adj_matrix, beta):
    """Compute the diffusion kernel using matrix exponential."""
    laplacian = get_laplacian_matrix(adj_matrix)
    return expm(-beta * laplacian)

# Parameters
num_nodes = 50
# Generate an undirected random graph
probability = 0.1  # Probability of edge creation
G = nx.erdos_renyi_graph(num_nodes, probability, directed=False)  # Ensure the graph is undirected
adjacency_matrix = nx.to_numpy_array(G)  # Convert to adjacency matrix

# Generate noisy samples function
def generate_noisy_samples(beta_sample):
    K = diffusion_kernel(adjacency_matrix, beta_sample)
    L = np.linalg.cholesky(K + 1e-6 * np.eye(num_nodes))  # Cholesky decomposition
    true_samples = L @ np.random.normal(size=(num_nodes, 1))  # Sample from Gaussian process
    noise = 0.05 * np.random.randn(num_nodes, 1)  # Additive noise
    Y_noisy = true_samples + noise  # Noisy observations
    return Y_noisy

class TaylorExpansionDiffusionKernel(gpflow.kernels.Kernel):
    def __init__(self, adjacency_matrix, num_terms=21, **kwargs):
        super().__init__(**kwargs)
        self.adjacency_matrix = tf.convert_to_tensor(adjacency_matrix, dtype=tf.float64)
        self.beta = gpflow.Parameter(2.0, transform=gpflow.utilities.positive())  # Learnable hyperparameter
        self.num_terms = num_terms  # Number of terms for Taylor expansion

    def K(self, X1, X2=None):
        if X2 is None:
            X2 = X1
        # Compute the Taylor expansion kernel matrix
        kernel_matrix = self.taylor_expansion_diffusion_kernel(self.adjacency_matrix, self.beta)
        indices_X1 = tf.cast(tf.reshape(X1, [-1]), dtype=tf.int32)
        indices_X2 = tf.cast(tf.reshape(X2, [-1]), dtype=tf.int32)
        return tf.gather(tf.gather(kernel_matrix, indices_X1, axis=0), indices_X2, axis=1)

    def K_diag(self, X):
        kernel_matrix = self.taylor_expansion_diffusion_kernel(self.adjacency_matrix, self.beta)
        indices_X = tf.cast(tf.reshape(X, [-1]), dtype=tf.int32)
        return tf.gather(tf.linalg.diag_part(kernel_matrix), indices_X)

    def get_laplacian_matrix(self, W):
        """Get the graph Laplacian from the adjacency matrix."""
        W = np.array(W)  # Ensure W is a numpy array
        nb_vertices = len(W)
        L = np.eye(nb_vertices) * np.sum(W, axis=1)
        for i in range(nb_vertices):
            for j in range(i):
                if W[i, j] > 0:
                    L[i, j] = L[j, i] = -W[i, j] / np.sqrt(np.sum(W[i]) * np.sum(W[j]))
        return L

    def taylor_expansion_diffusion_kernel(self, adj_matrix, beta):
        """Compute the Taylor expansion of the diffusion kernel."""
        laplacian = self.get_laplacian_matrix(adj_matrix)
        K = np.eye(adj_matrix.shape[0])  # Initialize kernel matrix
        for i in range(1, self.num_terms):
            K += (-beta)**i * np.linalg.matrix_power(laplacian, i) / math.factorial(i)
        
        # K += 1e-2 * np.eye(adj_matrix.shape[0]) # Adding a small epsilon for numerical stability

        return tf.convert_to_tensor(K, dtype=tf.float64)  # Convert to TensorFlow tensor


# Function to plot the results
def plot_results(beta_sample):
    clear_output(wait=True)  # Clear previous output
    # Generate noisy samples
    Y_noisy = generate_noisy_samples(beta_sample)

    # Convert noisy data to TensorFlow tensor
    X = tf.convert_to_tensor(np.arange(num_nodes, dtype=np.float64).reshape(-1, 1))  # Input features (nodes)
    Y = tf.convert_to_tensor(Y_noisy, dtype=tf.float64)  # Noisy sampled data

    # Create an instance of the kernel
    graph_kernel = TaylorExpansionDiffusionKernel(adjacency_matrix)

    # GPflow model
    model = gpflow.models.GPR(data=(X, Y), kernel=graph_kernel, mean_function=None)

    # Optimize the model
    gpflow.optimizers.Scipy().minimize(model.training_loss, model.trainable_variables)

    # Prediction for visualization
    X_new = tf.convert_to_tensor(np.arange(num_nodes).reshape(-1, 1), dtype=tf.float64)  # New input features for prediction
    mean, variance = model.predict_f(X_new)  # Predict mean and variance
    stddev = tf.sqrt(variance)  # Standard deviation

    # Create side-by-side plots for GP and network graph
    fig, ax = plt.subplots(1, 2, figsize=(16, 6))

    # Gaussian Process plot on the left
    ax[0].plot(X.numpy(), Y.numpy(), 'ro', label='Noisy Samples')
    ax[0].plot(X_new.numpy(), mean.numpy(), 'b-', label='Fitted Mean')
    ax[0].fill_between(X_new.numpy().flatten(),
                       (mean - 1.96 * stddev).numpy().flatten(),
                       (mean + 1.96 * stddev).numpy().flatten(),
                       color='lightblue', alpha=0.5, label='95% Confidence Interval')
    ax[0].set_title(f'Gaussian Process Fit with Graph Diffusion Kernel (beta={beta_sample:.2f})')
    ax[0].set_xlabel('Node Number')
    ax[0].set_ylabel('Sampled Value')
    ax[0].grid()
    ax[0].legend()
    
    # Network graph plot on the right
    plot_network_graph(adjacency_matrix, ax[1])

    plt.show()
    print("Learned beta:", model.kernel.beta.numpy())

# Function to plot the network graph in Matplotlib
def plot_network_graph(adjacency_matrix, ax):
    # Create a NetworkX graph from the adjacency matrix
    G = nx.from_numpy_array(adjacency_matrix)
    pos = nx.spring_layout(G)
    
    # Draw nodes
    nx.draw_networkx_nodes(G, pos, ax=ax, node_size=300, node_color='red')
    
    # Draw edges
    nx.draw_networkx_edges(G, pos, ax=ax, edge_color='blue')
    
    ax.set_title("Network Graph Representation of Adjacency Matrix")
    ax.axis("off")

# Create a slider for beta
beta_slider = widgets.FloatSlider(value=0.2, min=0.05, max=2.0, step=0.05, description='Beta:')
ui = widgets.VBox([beta_slider])

# Link the slider to the plotting function
out = widgets.interactive_output(plot_results, {'beta_sample': beta_slider})
display(ui, out)


Output()

### Combined Code

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm
import gpflow
import tensorflow as tf
import ipywidgets as widgets
from IPython.display import display, clear_output
import networkx as nx
import math

# Function to compute the exact diffusion kernel
def diffusion_kernel(adj_matrix, beta):
    laplacian = np.diag(np.sum(adj_matrix, axis=1)) - adj_matrix  # Graph Laplacian
    return expm(-beta * laplacian)  # Matrix exponential

# Generate noisy samples function
def generate_noisy_samples(adj_matrix, beta_sample):
    K = diffusion_kernel(adj_matrix, beta_sample)
    L = np.linalg.cholesky(K + 1e-6 * np.eye(adj_matrix.shape[0]))  # Cholesky decomposition
    true_samples = L @ np.random.normal(size=(adj_matrix.shape[0], 1))  # Sample from GP
    noise = 0.1 * np.random.randn(adj_matrix.shape[0], 1)  # Additive noise
    return true_samples + noise  # Noisy observations

# Taylor Expansion Diffusion Kernel class
class TaylorExpansionDiffusionKernel(gpflow.kernels.Kernel):
    def __init__(self, adjacency_matrix, num_terms=15, **kwargs):
        super().__init__(**kwargs)
        self.adjacency_matrix = tf.convert_to_tensor(adjacency_matrix, dtype=tf.float64)
        self.beta = gpflow.Parameter(2.0, transform=gpflow.utilities.positive())  # Learnable parameter
        self.num_terms = num_terms

    def K(self, X1, X2=None):
        if X2 is None:
            X2 = X1
        kernel_matrix = self.taylor_expansion_diffusion_kernel(self.adjacency_matrix, self.beta)
        indices_X1 = tf.cast(tf.reshape(X1, [-1]), dtype=tf.int32)
        indices_X2 = tf.cast(tf.reshape(X2, [-1]), dtype=tf.int32)
        return tf.gather(tf.gather(kernel_matrix, indices_X1, axis=0), indices_X2, axis=1)

    def K_diag(self, X):
        kernel_matrix = self.taylor_expansion_diffusion_kernel(self.adjacency_matrix, self.beta)
        indices_X = tf.cast(tf.reshape(X, [-1]), dtype=tf.int32)
        return tf.gather(tf.linalg.diag_part(kernel_matrix), indices_X)

    def get_laplacian_matrix(self, W):
        D = np.diag(np.sum(W, axis=1))
        return D - W

    def taylor_expansion_diffusion_kernel(self, adj_matrix, beta):
        laplacian = self.get_laplacian_matrix(adj_matrix)
        K = np.eye(adj_matrix.shape[0])
        for i in range(1, self.num_terms):
            K += (-beta)**i * np.linalg.matrix_power(laplacian, i) / math.factorial(i)
        return tf.convert_to_tensor(K, dtype=tf.float64)

# Exact Diffusion Kernel class
class GraphDiffusionKernel(gpflow.kernels.Kernel):
    def __init__(self, adjacency_matrix, **kwargs):
        super().__init__(**kwargs)
        self.adjacency_matrix = tf.convert_to_tensor(adjacency_matrix, dtype=tf.float64)
        self.beta = gpflow.Parameter(2.0, transform=gpflow.utilities.positive())

    def K(self, X1, X2=None):
        if X2 is None:
            X2 = X1
        kernel_matrix = diffusion_kernel(self.adjacency_matrix, self.beta.numpy())
        indices_X1 = tf.cast(tf.reshape(X1, [-1]), dtype=tf.int32)
        indices_X2 = tf.cast(tf.reshape(X2, [-1]), dtype=tf.int32)
        return tf.gather(tf.gather(kernel_matrix, indices_X1, axis=0), indices_X2, axis=1)

    def K_diag(self, X):
        kernel_matrix = diffusion_kernel(self.adjacency_matrix, self.beta.numpy())
        indices_X = tf.cast(tf.reshape(X, [-1]), dtype=tf.int32)
        return tf.gather(tf.linalg.diag_part(kernel_matrix), indices_X)

# Plotting function
def plot_results(beta_sample):
    clear_output(wait=True)
    Y_noisy = generate_noisy_samples(adjacency_matrix, beta_sample)
    X = tf.convert_to_tensor(np.arange(num_nodes, dtype=np.float64).reshape(-1, 1))
    Y = tf.convert_to_tensor(Y_noisy, dtype=tf.float64)

    # Taylor-expanded kernel GP model
    taylor_kernel = TaylorExpansionDiffusionKernel(adjacency_matrix)
    model_taylor = gpflow.models.GPR(data=(X, Y), kernel=taylor_kernel, mean_function=None)
    gpflow.optimizers.Scipy().minimize(model_taylor.training_loss, model_taylor.trainable_variables)

    # Exact diffusion kernel GP model
    exact_kernel = GraphDiffusionKernel(adjacency_matrix)
    model_exact = gpflow.models.GPR(data=(X, Y), kernel=exact_kernel, mean_function=None)
    gpflow.optimizers.Scipy().minimize(model_exact.training_loss, model_exact.trainable_variables)

    # Predictions for both models
    X_new = tf.convert_to_tensor(np.arange(num_nodes).reshape(-1, 1), dtype=tf.float64)
    mean_taylor, var_taylor = model_taylor.predict_f(X_new)
    mean_exact, var_exact = model_exact.predict_f(X_new)
    stddev_taylor = tf.sqrt(var_taylor)
    stddev_exact = tf.sqrt(var_exact)

    # Plotting
    fig, ax = plt.subplots(1, 3, figsize=(18, 6))
    
    # Taylor-expanded kernel plot
    ax[0].plot(X.numpy(), Y.numpy(), 'ro', label='Noisy Samples')
    ax[0].plot(X_new.numpy(), mean_taylor.numpy(), 'b-', label='Fitted Mean')
    ax[0].fill_between(X_new.numpy().flatten(),
                       (mean_taylor - 1.96 * stddev_taylor).numpy().flatten(),
                       (mean_taylor + 1.96 * stddev_taylor).numpy().flatten(),
                       color='lightblue', alpha=0.5, label='95% CI')
    ax[0].set_title("Taylor-Expanded Kernel GP")
    ax[0].set_xlabel('Node Number')
    ax[0].set_ylabel('Sampled Value')
    ax[0].grid()
    ax[0].legend()

    # Exact diffusion kernel plot
    ax[1].plot(X.numpy(), Y.numpy(), 'ro', label='Noisy Samples')
    ax[1].plot(X_new.numpy(), mean_exact.numpy(), 'g-', label='Fitted Mean')
    ax[1].fill_between(X_new.numpy().flatten(),
                       (mean_exact - 1.96 * stddev_exact).numpy().flatten(),
                       (mean_exact + 1.96 * stddev_exact).numpy().flatten(),
                       color='lightgreen', alpha=0.5, label='95% CI')
    ax[1].set_title("Exact Diffusion Kernel GP")
    ax[1].set_xlabel('Node Number')
    ax[1].set_ylabel('Sampled Value')
    ax[1].grid()
    ax[1].legend()

    # Network graph plot
    plot_network_graph(adjacency_matrix, ax[2])

    plt.show()

# Network graph plot function
def plot_network_graph(adjacency_matrix, ax):
    G = nx.from_numpy_array(adjacency_matrix)
    pos = nx.spring_layout(G)
    nx.draw_networkx_nodes(G, pos, ax=ax, node_size=300, node_color='red')
    nx.draw_networkx_edges(G, pos, ax=ax, edge_color='blue')
    ax.set_title("Network Graph Representation")
    ax.axis("off")

# Parameters
num_nodes = 40
adjacency_matrix = np.eye(num_nodes, k=1) + np.eye(num_nodes, k=-1)  # Circular adjacency matrix

# Slider for beta parameter
beta_slider = widgets.FloatSlider(value=1.0, min=0.05, max=2.0, step=0.05, description='Beta:')
ui = widgets.VBox([beta_slider])

# Interactive output
out = widgets.interactive_output(plot_results, {'beta_sample': beta_slider})
display(ui, out)


Output()

### Other Notes

In [7]:
# G py

# GPflow

# GP Jax


# constained optimization - positive values - minimize() still works



# f (modulation function) exist  - postitive definite kernel